### Compare-kreport-taxonomic-profiles.ipynb

The goal of this Jupyter notebook is to facilitate easy comparisons of the taxonomic 
profiles of multiple metagenomic samples. These analyses require kraken-report 
(kreport) format files as inputs, and the analyses can be performed for any 
taxonomic rank in the kreport files (strain, species, genus, family, etc.). 

This notebook will also provide a summary of how many reads were assigned across taxonomic ranks, and the number of unique taxa assigned across taxonomic ranks.

Running the notebook requires the pandas, seaborn, and matplotlib Python libraries.

The kreport format is standard for many profilers (Kraken, Bracken, Centrifuge, MMSeqs2), and 
there are conversion scripts available for Metamaps, MetaPhlAn3, and MEGAN in:
https://github.com/PacificBiosciences/pb-metagenomics-tools/tree/master/pb-metagenomics-scripts
The PacBio MEGAN-LR workflows also produce kreport output formats:
https://github.com/PacificBiosciences/pb-metagenomics-tools/

To run your own analyses, you will need to edit the sample/file path information. 
You can generate counts at different taxnomic levels using the get_all_dfs() function.
There are examples of how to do this at the species and genus levels, along with
how to save the count tables and create/save stacked barplot figures. 

**To make high-quality plots, you will likely need to change aspects of the plot parameters. This will depend on the number of samples in your dataset as well as their microbial diversity. There are clear explanations in the species level analysis for how to change different plot parameters.**

Daniel Portik 

Bioinformatics Scientist, Pacific Biosciences

dportik@pacificbiosciences.com

05/2021

-------------------

# Table of Contents <a name="Top"></a>

+ [Entering Sample Information](#ESI)
+ [Functions for Interpreting kreports](#FIK)
+ [Read Utilization Analysis](#RUA)
    + [Plot Percent Reads per Rank](#PRR)
    + [Plot Unique Taxa per Rank](#PTR)
+ [Species Level Analysis](#SP)
    + [Creating dataframes](#SPDF)
    + [Stacked Barplots](#SPSB)
    + [Heatmaps](#SPH)
    + [Barplot with Absolute Counts](#SPAB)
+ [Genus Level Analysis](#GN)
    + [Creating dataframes](#GNDF)
    + [Stacked Barplots](#GNSB)
    + [Heatmaps](#GNH)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Enter sample and file information here <a name="ESI"></a>

In [ ]:
# Provide the full path to each kreport to include below. 
# In this example we are using four files. The variable name
# is ideally what you want to call the sample in the analysis.

BC_138274 = '/Users/dportik/Documents/Projects/Datasets/CID_138274.megan-RMA-c2c.kreport.txt'
BC_138390 = '/Users/dportik/Documents/Projects/Datasets/CID_138390.megan-RMA-c2c.kreport.txt'
BC_139369 = '/Users/dportik/Documents/Projects/Datasets/CID_139369.megan-RMA-c2c.kreport.txt'
BC_139445 = '/Users/dportik/Documents/Projects/Datasets/CID_139445.megan-RMA-c2c.kreport.txt'

# Create a dictionary with string labels for the files as keys and the 
# a list for the values. The first item in the list is the file path, 
# and the second item should be the number of reads in the sample.
# The string labels will be used to label the samples in the resulting 
# tables and figures.

# Follow the format below for your own data.

file_dict = {"BC_138274": [BC_138274, 1792146], 
             "BC_138390": [BC_138390, 1687238], 
             "BC_139369": [BC_139369, 1904159], 
             "BC_139445": [BC_139445, 1767289]}

[Back to Top](#Top)

### Functions for interpreting kreports <a name="FIK"></a>

In [ ]:
# The series of functions will create three types of pandas dataframes with taxon 
# counts for a specified taxonomic rank. The first dataframe contains absolute counts of 
# the taxa across all samples. It can be exported as a CSV file, and is also transformed to 
# make the second and third dataframes. These additional dataframes will contain the 
# absolute or normalized counts in a transposed format that is used to make the stacked
# barplot figures. Do not edit these functions!

def get_unique_taxa_at_rank(file_dict, rank):
    """
    Function to obtain all unique taxon names at a given rank 
    for a set of kreport files provided in file_dict. Returns 
    sorted list of unique taxon names.
    
    :param file_dict: dictionary with string label as key and file path as val
    :param rank: desired taxonomic rank (e.g., 'S1', 'S', 'G', 'F', etc.)
    :return: sorted list of unique taxon names at rank
    """
    # initiate empty set for taxon names
    taxon_set = set()
    
    # iterate over the file dictionary
    for key, val in file_dict.items():
        
        # create a pandas dataframe, remove whitespace to clean names column of kreport 
        df = pd.read_csv(val[0], sep = '\t', header=None, skipinitialspace=True,
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])
        
        # get all unique taxon names at rank, add to set
        taxa = [t for t in (df.loc[df['rank'] == rank, 'name']).unique()]
        taxon_set.update(taxa)
    
    # return sorted list of taxon names
    return sorted(taxon_set)

def make_rank_dataframe(file_dict, rank):
    """
    Function to create a dataframe with taxon counts for 
    all kreport files in file_dict at a given taxonomic rank. 
    Returns pandas dataframe.
    
    :param file_dict: dictionary with string label as key and file path as val
    :param rank: desired taxonomic rank (e.g., 'S1', 'S', 'G', 'F', etc.)
    :return: pandas dataframe of taxon counts for specified rank
    """
    
    # initiate empty dictionary
    taxon_count_dict = {}
    
    # get taxon names to use for this rank
    taxon_list = get_unique_taxa_at_rank(file_dict, rank)
    
    # iterate over the file dictionary
    for key, val in file_dict.items():
        
        # add this sample as a key in the taxon count dictionary
        # value is an empty list that will be filled with taxon counts
        # in the order of the taxon_list
        taxon_count_dict[key] = []
        
        # make temp dataframe, remove whitespace to clean names column of kreport 
        df = pd.read_csv(val[0], sep = '\t', header=None, skipinitialspace=True, 
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])
        
        # iterate over taxon names 
        for taxon in taxon_list:
            # make a subframe with matched name
            temp = df.loc[df['name'] == taxon]
            # add the cumulative count for the taxon name to the dict list
            # this should only be a single name, taking sum as easy way to get integer
            taxon_count_dict[key].append(temp.loc[df['rank'] == rank, 'cumulative_count'].sum())
    
    # create new dataframe from the dictionaries constructed
    df_taxa = pd.DataFrame.from_dict(taxon_count_dict)
    
    # add in a column with the taxon labels
    df_taxa.insert(0, 'Taxa', taxon_list)
    
    # return the taxon dataframe
    return df_taxa

def prepare_plotting_dfs(df, file_dict, sortby=list(file_dict.keys())):
    # transpose dataframe, sort counts, and add new column names
    df_transpose = df.sort_values(by=sortby, ascending=False).set_index('Taxa').transpose().rename_axis('Samples', axis=1)
    # normalize the species counts of each sample 100%
    df_transpose_norm = df_transpose.apply(lambda x: x*100/sum(x), axis=1)
    
    return df_transpose, df_transpose_norm

def get_all_dfs(file_dict, rank, sortby=list(file_dict.keys())):
    df_one = make_rank_dataframe(file_dict, rank)
    df_two, df_three = prepare_plotting_dfs(df_one, file_dict, sortby=sortby)
    return df_one, df_two, df_three
    
    
##############################################################################
# This function will count the number of reads assigned at each rank level,
# as well as the number of unique taxa occurring at each rank level.


def get_reads_to_ranks(file_dict):
    
    rank_read_count_dict, rank_taxa_count_dict = {}, {}
    
    for key, val in file_dict.items():
        # insert dict key with empty list
        rank_read_count_dict[key] = []
        rank_taxa_count_dict[key] = []

        # create dataframe
        df = pd.read_csv(val[0], sep = '\t', header=None,
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])

        # iterate over desired ranks
        for r in ['S1', 'S', 'G', 'F', 'O', 'C', 'P', 'K']:

            # get number of reads assigned to all taxa at this rank level
            # we use level counts rather than cumulative counts here
            rank_readtotal = round((df.loc[df['rank'] == r, 'level_count'].sum() / val[1]) * 100, 1)
            # add to dictionary list
            rank_read_count_dict[key].append(rank_readtotal)

            # get number of unique names per rank
            rank_namecount = (df['rank'] == r).sum()
            # add to dictionary list
            rank_taxa_count_dict[key].append(rank_namecount)
            
        # convert dictionary to dataframe and insert new column in with rank names
    # do for read counts
    df_rank_read_count = pd.DataFrame.from_dict(rank_read_count_dict)
    df_rank_read_count.insert(0, 'Rank', ['S1', 'S', 'G', 'F', 'O', 'C', 'P', 'K'])

    # same for taxa counts
    df_rank_taxa_count = pd.DataFrame.from_dict(rank_taxa_count_dict)
    df_rank_taxa_count.insert(0, 'Rank', ['S1', 'S', 'G', 'F', 'O', 'C', 'P', 'K'])
    
    return df_rank_read_count, df_rank_taxa_count

[Back to Top](#Top)

# Read Utilization Analysis <a name="RUA"></a>

In [ ]:
# Here we are attempting to understand how many reads were assigned at particular
# taxonomic ranks, and how many unique taxa were inferred at different ranks.

# Use the get_reads_to_ranks() function to return the dataframes for plotting.
# You won't need to change anything here.
df_rank_read_count, df_rank_taxa_count = get_reads_to_ranks(file_dict)

[Back to Top](#Top)

### Plot percent of reads assigned across ranks <a name="PRR"></a>

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# do a quick manipulation of the dataframe for correct plotting orientation
df_rank_read_count_it = df_rank_read_count.set_index('Rank').transpose().rename_axis('Sample', axis=1)


# Create the horizontal stacked bar plot using the pandas .plot.barh() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we are setting a seaborn palette here
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor
# ylim: set limits for y-axis

ax = df_rank_read_count_it.plot.bar(stacked=True, figsize=(4,7), width=0.8, color=sns.color_palette("Spectral_r",8), 
                                    fontsize='large', edgecolor='black', linewidth=0.5,
                                    title='Percent-Reads-per-Rank', ylim=(0,100))

# for vertical plot, we need to reverse the legend labels, start by getting these objects
handles, labels = ax.get_legend_handles_labels()

# add reversed(handles) and reversed(labels) to the legend arguments
# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1,1), 
          fontsize='large', ncol=1, labelspacing=0.3)

# adjust the appearance of the x-labels
ax.set_xticklabels(df_rank_read_count_it.index, rotation=45, ha='right')

# Use this to save the plot to file
ax.figure.savefig('Percent-Reads-per-Rank.pdf', bbox_inches='tight')

[Back to Top](#Top)

### Plot number of taxa assigned across ranks <a name="PTR"></a>

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# do a quick manipulation of the dataframe for correct plotting orientation
df_rank_taxa_count_it = df_rank_taxa_count.set_index('Rank').transpose().rename_axis('Sample', axis=1)



# Create the horizontal stacked bar plot using the pandas .plot.barh() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we are setting a seaborn palette here
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor

ax = df_rank_taxa_count_it.plot.bar(stacked=True, figsize=(4,7), width=0.8, color=sns.color_palette("gist_stern_r",8), 
                                    fontsize='large', edgecolor='black', linewidth=0.5,
                                    title='Percent-Reads-per-Rank')

# for vertical plot, we need to reverse the legend labels, start by getting these objects
handles, labels = ax.get_legend_handles_labels()

# add reversed(handles) and reversed(labels) to the legend arguments
# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1,1), 
          fontsize='large', ncol=1, labelspacing=0.3)

# adjust the appearance of the x-labels
ax.set_xticklabels(df_rank_taxa_count_it.index, rotation=45, ha='right')

# Use this to save the plot to file
ax.figure.savefig('Percent-Taxa-per-Rank.pdf', bbox_inches='tight')

[Back to Top](#Top)

# Analysis at the SPECIES level <a name="SP"></a>

### SPECIES: Create species-level dataframes <a name="SPDF"></a>

In [ ]:
# This demonstrates how to use the main function get_all_dfs() to create 
# the comparison dataframes. We will start the demonstration for the 
# species level.

# To generate the dataframes, we use the following function: 
# get_all_dfs(file_dict, rank, sortby='Name')
#    the first positional argument is file_dict (leave unchanged)
#    the second positional argument is the rank desired ('S1', 'S', 'G', 'F', etc.)
#    the optional argument sortby can be used to specify one of the samples to sort 
#        abundances by, which must be a key name in file_dict
# The function returns three dataframes, so three variable names must be specified

# Here is the basic usage to get dataframes for species level taxa:
df_sp, df_sp_plot, df_sp_plot_norm = get_all_dfs(file_dict, 'S')
# In the above example, we labeled the three dataframes returned by the function as:
# df_sp, df_sp_plot, and df_sp_plot_norm.

# Below is the same example but with sorting by abundances in sample "CID_139445", un-hash to run:
# df_sp, df_sp_plot, df_sp_plot_norm = get_all_dfs(file_dict, 'S', sortby="CID_139445")
# Make sure if you use the optional argument that you specify a label name/key in the file_dict.

In [ ]:
# Preview the first type of dataframe. Here, samples are the columns and taxa form the rows. 
df_sp.head()

In [ ]:
# Preview the second dataframe. Notice it is a transposed version of the first one.
# Here, taxa are columns and samples are the rows.
df_sp_plot.head()

In [ ]:
# Preview the third dataframe. This is the same format as above,
# but the values have been normalized so counts for each sample 
# total to 100. 
df_sp_plot_norm.head()

### SPECIES: Saving the dataframes to files

In [ ]:
# to save any of the above dataframes, use the df.to_csv() method

# for the df_sp, we don't want to include the index (which is just integers)
# we can also specify tab-delimited using the sep argument:
df_sp.to_csv("Species-Absolute-Abundance-Counts.txt", index=False, sep='\t')

# for the df_sp_plot and df_sp_plot_norm dataframes, keep the index (which is the sample names):
df_sp_plot.to_csv("Species-Absolute-Abundance-Counts-Transposed.txt", sep='\t')


[Back to Top](#Top)

### SPECIES stacked barplots <a name="SPSB"></a>

#### Make a HORIZONTAL stacked barplot using the NORMALIZED COUNT dataframe. 

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# Create the horizontal stacked bar plot using the pandas .plot.barh() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we refer to pal which was set above
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor

ax = df_sp_plot_norm.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=sns.color_palette("tab20"), fontsize='large', 
                               edgecolor='black', linewidth=0.5)

# these commands eliminate the bounding box for the barplot
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(bbox_to_anchor=(1,-0.10), fontsize='x-small', ncol=7, labelspacing=0.3)

# You can save the plot using the following:
ax.figure.savefig('Species-Abundances-Normalized-Horizontal.pdf', bbox_inches='tight')

[Back to Top](#Top)

#### Make a VERTICAL stacked barplot using the NORMALIZED COUNT dataframe. 

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# Create the vertical stacked bar plot using the pandas .plot.bar() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we refer to pal which was set above
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor

ax = df_sp_plot_norm.plot.bar(stacked=True, figsize=(5,15), width=0.8, 
                              color=sns.color_palette("tab20"), fontsize='large', 
                              edgecolor='black', linewidth=0.5)

# these commands eliminate the bounding box for the barplot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# for vertical plot, we need to reverse the legend labels, start by getting these objects
handles, labels = ax.get_legend_handles_labels()

# and add reversed(handles) and reversed(labels) to the legend arguments
# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1,0.9), 
          fontsize='x-small', ncol=1, labelspacing=0.3)

# You can save the above plot using the following:
ax.figure.savefig('Species-Abundances-Normalized-Vertical.pdf', bbox_inches='tight')

[Back to Top](#Top)

### SPECIES heatmaps <a name="SPH"></a>

#### Make a HORIZONTAL heatmap using the NORMALIZED COUNT dataframe. 

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# set the size of the figure first
fig, ax = plt.subplots(figsize=(20, 2))

# use seaborn to generate the heatmap, set color to Blues
sns.heatmap(df_sp_plot_norm, cmap="Blues", linewidth=0.1, robust=True)

# these commands clean up aspects of the tick labels
ax.set_yticklabels(df_sp_plot_norm.index, rotation=0, ha='right', fontsize='medium')
ax.set_xticklabels(df_sp_plot_norm, rotation=45, ha='right', fontsize='x-small')

# show the plot
plt.show()

# You can save the plot using the following:
ax.figure.savefig('Species-Heatmap-Horizontal.pdf', bbox_inches='tight')

[Back to Top](#Top)

#### Make a VERTICAL heatmap using the NORMALIZED COUNT dataframe.

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# set the size of the figure first
fig, ax = plt.subplots(figsize=(4, 16))

# use seaborn to generate the heatmap, set color to Blues
sns.heatmap(df_sp_plot_norm.transpose(), cmap="Blues", linewidth=0.1, robust=True)

# these commands clean up aspects of the tick labels
ax.set_xticklabels(df_sp_plot_norm.transpose(), rotation=45, ha='right', fontsize='small')

# show the plot
plt.show()

# You can save the plot using the following:
ax.figure.savefig('Species-Heatmap-Vertical.pdf', bbox_inches='tight')

[Back to Top](#Top)

### Make a HORIZONTAL stacked barplot using the ABSOLUTE COUNT dataframe. <a name="SPAB"></a>

This is probably not a very useful plot if your samples have different numbers of reads, but it is provided as a demonstration.

In [ ]:
# In this plot, we don't expect the stacked bars to be the same lengths.

# All the same arguments are used, but we use the df_sp_plot dataframe instead
# of the df_sp_plot_norm dataframe

pal = sns.color_palette("tab20")
ax = df_sp_plot.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                          color=sns.color_palette("tab20"), fontsize='large', 
                          edgecolor='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(bbox_to_anchor=(1,-0.10), fontsize='x-small', ncol=7, labelspacing=0.3)

# You can save the above plot using the following:
ax.figure.savefig('Species-Abundances-Absolute-Horizontal.pdf', bbox_inches='tight')

[Back to Top](#Top)

# Analysis at the GENUS level <a name="GN"></a>

### Create a taxon dataframe for the GENUS level <a name="GNDF"></a>

In [ ]:
# Do the same thing, but for the genus level

# here is the basic usage to get dataframes for genus level taxa
df_gn, df_gn_plot, df_gn_plot_norm = get_all_dfs(file_dict, 'G')


In [ ]:
# Preview the first dataframe. Here, samples are the columns and taxa form the rows.
df_gn.head()

In [ ]:
# Save genus counts to csv
df_gn.to_csv("Genus-Absolute-Abundance-Counts.txt", index=False, sep='\t')

[Back to Top](#Top)

### GENUS stacked barplots <a name="GNSB"></a>

#### Make a HORIZONTAL stacked barplot using the NORMALIZED COUNT dataframe.

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

ax = df_gn_plot_norm.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=sns.color_palette("tab20"), fontsize='large', 
                               edgecolor='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(bbox_to_anchor=(1,-0.10), fontsize='small', ncol=7, labelspacing=0.1)
ax.figure.savefig('Genus-Abundances-Normalized-Horizontal.pdf', bbox_inches='tight')

[Back to Top](#Top)

#### Make a VERTICAL stacked barplot using the NORMALIZED COUNT dataframe. 

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

ax = df_gn_plot_norm.plot.bar(stacked=True, figsize=(5,15), width=0.8, 
                              color=sns.color_palette("tab20"), fontsize='large', 
                              edgecolor='black', linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
handles, labels = ax.get_legend_handles_labels()
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1,0.9), 
          fontsize='small', ncol=1, labelspacing=0.3)
ax.figure.savefig('Genus-Abundances-Normalized-Vertical.pdf', bbox_inches='tight')

[Back to Top](#Top)

### GENUS heatmaps <a name="GNH"></a>

#### Make a HORIZONTAL heatmap using the NORMALIZED COUNT dataframe. <a name="GNHH"></a>

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

fig, ax = plt.subplots(figsize=(20, 2))
sns.heatmap(df_gn_plot_norm, cmap="Blues", linewidth=0.1, robust=True)
ax.set_yticklabels(df_gn_plot_norm.index, rotation=0, ha='right', fontsize='medium')
ax.set_xticklabels(df_gn_plot_norm, rotation=45, ha='right', fontsize='small')
plt.show()
ax.figure.savefig('Genus-Heatmap-Horizontal.pdf', bbox_inches='tight')

[Back to Top](#Top)

#### Make a VERTICAL heatmap using the NORMALIZED COUNT dataframe. <a name="GNVH"></a>

In [ ]:
# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

fig, ax = plt.subplots(figsize=(4, 10))
sns.heatmap(df_gn_plot_norm.transpose(), cmap="Blues", linewidth=0.1, robust=True)
ax.set_xticklabels(df_gn_plot_norm.transpose(), rotation=45, ha='right', fontsize='small')
plt.show()
ax.figure.savefig('Genus-Heatmap-Vertical.pdf', bbox_inches='tight')

[Back to Top](#Top)